In [1]:
import sys
sys.path.append('..')  # allow this notebook to find equal-level directories
%pylab inline
from importing_modules import *
# pyomo.environ as oe, seaborn as sns, plotly.plotly as py, plotly.graph_objs as go
# from util.gjh_wrapper import gjh_solve, make_df, from vis import acres_bars, zL_bars

Populating the interactive namespace from numpy and matplotlib


## Create problem instance

In [2]:
# Load data for each set, parameter, etc. to define a problem instance
objwrapper = LoadObj()
# lrsegs = ['N42071SL2_2410_2700']
lrsegs = ['N51133RL0_6450_0000']
data = objwrapper.load_data(savedata2file=False, lrsegs_list=lrsegs)

# Set the cost bound ----
data.totalcostupperbound = 100000
costboundstr = str(round(data.totalcostupperbound, 1))

# Create concrete problem instance using the separately defined optimization model
mdl = objwrapper.create_concrete(data=data)

# Retain only the Nitrogen load objective, and deactivate the others
mdl.PercentReduction['P'].deactivate()
mdl.PercentReduction['S'].deactivate()

# ---- Solver name ----
localsolver = True
solvername = 'ipopt'
# solvername = 'minos'

## Solve problem instance

In [3]:
df_list = []
solution_objectives = OrderedDict()
for ii in range(10):
    print(ii)
    
    # reinitialize the variables
    for k in mdl.x:
#         mdl.x[k] = float(random.randrange(0, 600001))/100
        mdl.x[k] = round(random.uniform(0, 6000), 2)
    
    # print the cost bound values (C)
    costboundstr = str(round(data.totalcostupperbound, 1))
    print(costboundstr)
    
    # Set names for saving output
    looptimestamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    loopname = ''.join(['output/loadobj_startingpoint', str(ii),
                        '_costbound', costboundstr, '_', solvername, '_',
                           looptimestamp])
        
    # Solve The Model
    myobj = SolveAndParse(instance=mdl, data=data, localsolver=localsolver, solvername=solvername)
    logfilename = os.path.join(projectpath, ''.join([loopname, '.txt']))
    # set filepath for saving information about all of the solver iterates
    myobj.modify_ipopt_options(optionsfilepath='ipopt.opt',
                               newoutputfilepath=os.path.join(projectpath, ''.join([loopname, '.iters'])))
    merged_df = myobj.solve(logfilename=logfilename)
    print('shape of df is: %s' % str(merged_df.shape))
    print('\nObjective is: %d' % oe.value(mdl.PercentReduction['N']))
    
    # Save this run's objective value in a list
    solution_objectives[mdl.totalcostupperbound.value] = oe.value(mdl.PercentReduction['N'])
    merged_df['solution_objectives'] = oe.value(mdl.PercentReduction['N'])
    
    # Label this run in the dataframe
    merged_df['totalcostupperbound'] = mdl.totalcostupperbound.value
    merged_df['startpointiterate'] = ii
    sorteddf_byacres = merged_df.sort_values(by='acres')
    
#     # ---- Make zL Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '_zL', .png']))

#     zL_bars(df=merged_df, instance=mdl,
#             savefig=True, savefilepathandname=savefilepathandname)
    
#     # ---- Acres Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '_x', .png']))

#     objstr = ''.join(['Objective is: ', str(round(oe.value(mdl.PercentReduction['N']),2))])
#     coststr = ''.join(['Total cost is: ', str(round(oe.value(mdl.Total_Cost.body),1))])
#     titlestr = '\n'.join([objstr, coststr, 'labels are (cost per unit, total bmp instance cost)'])

#     acres_bars(df=sorteddf_byacres, instance=mdl, titlestr=titlestr,
#                savefig=True, savefilepathandname=savefilepathandname)    
    
#     # -- Gradient, Jacobian, Hessian
#     gjh_filename, g = gjh_solve(instance=mdl,
#                             keepfiles=True,
#                             amplenv=ampl,
#                             basegjhpath=os.getcwd())

#     g_df = make_df(instance=mdl, filterbydf=merged_df, g=g)

#     sorteddf_byacres = sorteddf_byacres.merge(g_df, how='left',
#                                   on=['bmpshortname', 'landriversegment', 'loadsource'],
#                                   sort=False)
    
#     # ---- Make gradient Figure ----
#     keystrs = [str([x, y]) for x, y in zip(sorteddf_byacres['bmpshortname'], sorteddf_byacres['loadsource'])]
#     # Make Figure
#     fig = plt.figure(figsize=(10, 4))
#     rects = plt.barh(y=keystrs, width=sorteddf_byacres['g'])
#     ax = plt.gca()

#     ax.set_position([0.3,0.1,0.5,0.8])

#     plt.savefig(os.path.join(projectpath, ''.join([loopname, '_g', .png'])))
    
    
    
# Save all of the solutions in a list
df_list.append(sorteddf_byacres)

# Save the results to a .csv file
alldfs = pd.concat(df_list, ignore_index=True)
alldfs['x'] = list(zip(alldfs.bmpshortname, alldfs.landriversegment, alldfs.loadsource, alldfs.totalannualizedcostperunit))
filenamestr = ''.join(['output/loadobj_difstartpts_alldfs', '_', solvername, '_',
                           datetime.now().strftime('%Y-%m-%d_%H%M%S'), '.csv'])
alldfs.to_csv(os.path.join(projectpath, filenamestr))

0
100000
Solver log file: '/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/ampl/OptEfficiencySubProblem/output/loadobj_startingpoint0_costbound100000_ipopt_2018-08-09_124314.txt'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpzifw17f3.pyomo.sol'
Solver problem files: ('/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpzifw17f3.pyomo.nl',)
Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of

Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2628
Number of nonzeros in Lagrangian Hessian.............:     5278

Total number of variables............................:     1339
                     variables with only lower bounds:     1339
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

shape of df is: (14, 7)

Objective is: 6
2
100000
    'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
    (type=<class 'pyomo.core.base.suffix.Suffix'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.suffix.Suffix'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
Solver log file: '/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/a

  61 -6.8166854e+00 0.00e+00 1.23e+00  -8.6 2.35e+00    -  9.71e-01 1.00e+00f  1
  62 -6.8166855e+00 0.00e+00 2.54e-14  -8.6 5.62e-01    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 62

                                   (scaled)                 (unscaled)
Objective...............:  -6.8166854703458268e+00   -6.8166854703458268e+00
Dual infeasibility......:   2.5368596112684827e-14    2.5368596112684827e-14
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   9.4536629107631268e-09    9.4536629107631268e-09
Overall NLP error.......:   9.4536629107631268e-09    9.4536629107631268e-09


Number of objective function evaluations             = 69
Number of objective gradient evaluations             = 63
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 69
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 63
Num

  49 -6.7834113e+00 0.00e+00 3.55e+04  -8.6 5.30e+01    -  7.01e-01 9.99e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50 -6.7875146e+00 0.00e+00 2.02e+04  -8.6 8.47e+01    -  4.30e-01 9.79e-01f  1
  51 -6.8084952e+00 0.00e+00 6.64e+03  -8.6 1.57e+01    -  6.72e-01 1.00e+00f  1
  52 -6.8153566e+00 0.00e+00 1.98e+03  -8.6 4.34e+00    -  7.02e-01 1.00e+00f  1
  53 -6.8154831e+00 0.00e+00 1.17e+03  -8.6 2.18e+01    -  4.09e-01 1.00e+00f  1
  54 -6.8166640e+00 0.00e+00 3.36e+02  -8.6 1.89e+00    -  7.12e-01 1.00e+00f  1
  55 -6.8166790e+00 0.00e+00 1.41e+02  -8.6 6.84e+00    -  5.81e-01 9.97e-01f  1
  56 -6.8166838e+00 0.00e+00 4.26e+01  -8.6 7.45e+00    -  6.98e-01 1.00e+00f  1
  57 -6.8166854e+00 0.00e+00 1.23e+00  -8.6 2.35e+00    -  9.71e-01 1.00e+00f  1
  58 -6.8166854e+00 0.00e+00 2.54e-14  -8.6 5.62e-01    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 58

                                   (scaled)                 (unscaled)
Objecti

...

  24 -4.6250029e+00 2.96e+05 4.94e+04  -1.0 5.71e+03    -  5.41e-02 5.48e-01h  1
  25 -3.0964610e+00 1.20e+05 1.00e+05  -1.0 3.12e+03    -  1.29e-01 5.94e-01h  1
  26 -2.9509613e+00 6.74e+04 1.57e+05  -1.0 1.29e+03    -  1.96e-01 4.39e-01h  1
  27 -2.9053255e+00 5.01e+04 6.56e+04  -1.0 4.25e+02    -  3.29e-01 2.57e-01h  1
  28 -2.8705906e+00 3.35e+04 1.28e+05  -1.0 3.20e+02    -  2.37e-01 3.30e-01h  1
  29 -2.8611541e+00 2.53e+04 2.89e+05  -1.0 2.24e+02    -  5.51e-01 2.45e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30 -2.8692753e+00 1.51e+04 4.71e+05  -1.0 2.15e+02    -  6.01e-01 4.01e-01h  1
  31 -2.9138644e+00 8.96e+03 1.36e+06  -1.0 2.03e+02    -  9.44e-01 4.06e-01h  1
  32 -3.5428467e+00 2.57e+03 7.88e+05  -1.0 1.79e+03    -  8.83e-01 7.05e-01h  1
  33 -3.6256558e+00 1.56e+03 2.16e+06  -1.0 3.70e+02    -  1.00e+00 3.74e-01h  1
  34 -3.7396071e+00 2.09e+02 6.31e+05  -1.0 2.48e+02    -  1.00e+00 8.02e-01h  1
  35 -3.7486748e+00 9.75e+01

  15 -9.7337067e+00 2.15e+06 7.94e+03  -1.0 1.05e+03    -  2.24e-02 2.85e-01h  1
  16 -9.5379676e+00 1.70e+06 9.66e+03  -1.0 1.65e+03    -  6.77e-02 2.10e-01h  1
  17 -9.2278059e+00 1.29e+06 1.30e+04  -1.0 3.69e+03    -  5.54e-02 2.42e-01h  1
  18 -8.8833949e+00 1.05e+06 1.54e+04  -1.0 6.79e+03    -  6.30e-02 1.85e-01h  1
  19 -8.7392393e+00 9.95e+05 1.51e+04  -1.0 1.05e+04    -  4.31e-02 5.37e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20 -8.6857054e+00 8.48e+05 2.01e+04  -1.0 3.51e+03    -  7.41e-03 1.47e-01h  1
  21 -8.6537419e+00 7.93e+05 1.95e+04  -1.0 4.74e+03    -  5.24e-02 6.50e-02h  1
  22 -8.6232408e+00 7.73e+05 2.01e+04  -1.0 8.80e+03    -  8.83e-03 2.52e-02h  1
  23 -8.5265399e+00 7.33e+05 2.18e+04  -1.0 1.35e+04    -  1.04e-02 5.21e-02h  1
  24 -7.9903992e+00 6.57e+05 2.59e+04  -1.0 7.51e+03    -  1.25e-02 1.04e-01h  1
  25 -2.9837770e+00 1.83e+05 5.75e+04  -1.0 4.51e+03    -  7.43e-02 7.22e-01h  1
  26 -2.7860829e+00 1.19e+05

   3 -2.3304020e+01 9.55e+07 2.67e+01  -1.0 4.27e+02    -  8.19e-01 6.74e-01h  1
   4 -1.7212016e+01 4.53e+07 2.60e+02  -1.0 2.08e+03    -  1.53e-02 5.25e-01h  1
   5 -1.5235180e+01 2.95e+07 5.12e+02  -1.0 7.49e+03    -  4.54e-02 3.50e-01h  1
   6 -1.4154410e+01 2.10e+07 7.47e+02  -1.0 4.88e+03    -  6.74e-02 2.88e-01h  1
   7 -1.3707768e+01 1.78e+07 8.16e+02  -1.0 2.02e+03    -  7.74e-02 1.53e-01h  1
   8 -1.3278192e+01 1.51e+07 9.01e+02  -1.0 1.61e+03    -  7.78e-02 1.50e-01h  1
   9 -1.2817577e+01 1.29e+07 1.20e+03  -1.0 2.11e+03    -  1.93e-02 1.49e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -1.2283697e+01 1.03e+07 1.52e+03  -1.0 2.35e+03    -  6.08e-02 2.03e-01h  1
  11 -1.1670676e+01 7.45e+06 2.27e+03  -1.0 1.59e+03    -  4.02e-02 2.74e-01h  1
  12 -1.1181068e+01 5.64e+06 2.27e+03  -1.0 1.31e+03    -  1.62e-01 2.43e-01h  1
  13 -1.0099203e+01 3.20e+06 4.94e+03  -1.0 4.70e+03    -  2.96e-02 4.33e-01h  1
  14 -9.7836010e+00 2.58e+06

   3 -2.3825111e+01 1.04e+08 2.67e+01  -1.0 7.39e+02    -  9.90e-01 7.69e-01h  1
   4 -1.7585379e+01 5.18e+07 2.04e+02  -1.0 1.83e+03    -  7.68e-02 5.02e-01h  1
   5 -1.5492505e+01 3.47e+07 4.68e+02  -1.0 7.64e+03    -  7.72e-03 3.31e-01h  1
   6 -1.4813376e+01 2.93e+07 4.04e+02  -1.0 2.02e+03    -  1.49e-01 1.54e-01h  1
   7 -1.3526614e+01 1.95e+07 8.60e+02  -1.0 4.56e+03    -  7.52e-03 3.33e-01h  1
   8 -1.2930863e+01 1.56e+07 7.80e+02  -1.0 2.53e+03    -  1.68e-01 2.03e-01h  1
   9 -1.2320710e+01 1.27e+07 1.15e+03  -1.0 1.82e+03    -  3.25e-02 1.84e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -1.1580579e+01 9.06e+06 1.96e+03  -1.0 1.98e+03    -  1.61e-02 2.88e-01h  1
  11 -1.1071466e+01 6.93e+06 2.49e+03  -1.0 1.75e+03    -  6.54e-02 2.35e-01h  1
  12 -1.0604718e+01 4.88e+06 3.17e+03  -1.0 1.28e+03    -  1.12e-01 2.96e-01h  1
  13 -1.0302295e+01 3.92e+06 2.75e+03  -1.0 1.53e+03    -  1.77e-01 1.95e-01h  1
  14 -9.9429912e+00 2.73e+06

   3 -2.1981243e+01 7.35e+07 1.64e+01  -1.0 5.03e+02    -  3.95e-01 4.45e-01h  1
   4 -1.7513993e+01 4.21e+07 2.88e+02  -1.0 1.99e+03    -  6.26e-03 4.27e-01h  1
   5 -1.4918950e+01 2.42e+07 5.79e+02  -1.0 4.73e+03    -  6.64e-02 4.24e-01h  1
   6 -1.3961427e+01 1.81e+07 6.47e+02  -1.0 3.51e+03    -  1.46e-01 2.53e-01h  1
   7 -1.3390969e+01 1.52e+07 7.21e+02  -1.0 2.02e+03    -  9.32e-02 1.59e-01h  1
   8 -1.2714397e+01 1.24e+07 1.07e+03  -1.0 2.02e+03    -  3.19e-02 1.82e-01h  1
   9 -1.1881460e+01 8.70e+06 1.82e+03  -1.0 1.89e+03    -  2.48e-02 3.01e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -1.1284943e+01 6.46e+06 2.35e+03  -1.0 1.45e+03    -  7.94e-02 2.57e-01h  1
  11 -1.0715457e+01 4.61e+06 3.18e+03  -1.0 1.51e+03    -  8.62e-02 2.87e-01h  1
  12 -1.0212265e+01 3.74e+06 3.00e+03  -1.0 5.11e+03    -  1.50e-01 1.89e-01h  1
  13 -9.5697522e+00 2.80e+06 4.69e+03  -1.0 6.52e+03    -  6.12e-02 2.49e-01h  1
  14 -9.3118162e+00 1.86e+06

In [4]:
# solver = oe.SolverFactory('ipopt')
# results = solver.solve(mdl, tee=True, symbolic_solver_labels=True,
#                                    keepfiles=False)

In [5]:
i=0
for v in mdl.component_data_objects(oe.Var):
    if v.stale:
        continue
    i+=1
#     print(str(v), ' = ', value(v))
    
print(i)

1339


In [6]:
# for k in mdl.x.keys():
#     print(mdl.x[k].stale)

In [7]:
from util.solution_wrangler import get_nonzero_var_names_and_values
nzvnames, nzvvalues = get_nonzero_var_names_and_values(mdl, onlynotstale=True)

In [8]:
print(len(nzvnames))
nzvnames[1]

14


'x[UrbanNMPlanHR,N51133RL0_6450_0000,nch]'

In [9]:
from util.solution_wrangler import get_nonzero_var_df
nonzerodf = get_nonzero_var_df(mdl, addcosttbldata=data.costsubtbl)

In [10]:
print(nonzerodf.shape)
nonzerodf.head(5)

(14, 6)


,acres,bmpshortname,landriversegment,loadsource,totalannualizedcostperunit,totalinstancecost
0,1450.534430,CoverCropTradRED,N51133RL0_6450_0000,gom,68.94,99999.843578
1,2023.723265,UrbanNMPlanHR,N51133RL0_6450_0000,nch,0.00,0.000000
2,2834.603953,UrbanNMPlanHR,N51133RL0_6450_0000,ntg,0.00,0.000000
3,32.486103,UrbanNMPlanHR,N51133RL0_6450_0000,mcn,0.00,0.000000
4,3524.195328,HRTill,N51133RL0_6450_0000,dbl,0.00,0.000000


In [11]:
# # Other ways to access the optimal values:
# mdl.x['HRTill', 'N51133RL0_6450_0000', 'oac'].value
#
# tol = 1e-6
# for b in mdl.BMPS:
#     for lmbda in mdl.LOADSRCS:
#         bval = mdl.x[b, 'N51133RL0_6450_0000', lmbda].value
#         if not not bval:
#             if abs(bval)>tol:
#                 print('(%s, %s): %d' % (b, lmbda, bval))